In [18]:
import pyodbc
import os, json
import teradatasql
import pandas as pd
import datetime
from datetime import datetime, timedelta

arquivo_json = os.path.join(os.environ['USERPROFILE'], "config_robos.json")

with open(arquivo_json, encoding='utf-8') as f:
    dados = json.load(f)

usuarioID = dados["senha_rede"]["chave"]
senhaAcesso = dados["senha_rede"]["senha"]


import pyodbc

# Defina suas credenciais e propriedades de conexão
host = "vdados.petrobras.com.br"
port = 9401
username = usuarioID
password = senhaAcesso
database = "S10267"  # O dataSource

connection = pyodbc.connect('dsn=TDVCPBIP')

# Calcula a data formatada como hoje - 35 dias
dataFormatada = (datetime.now() - timedelta(days=35)).strftime('%Y-%m-%d')

strSQL = f"""
SELECT ESQUEMA.OBJNR AS OBJETO_ORDEM, 
    STD.TXT04 AS STATUS, 
    STD.TXT30 AS TEXTO_STATUS, 
    ST.USNAM AS USUARIO_MODIFICACAO_STATUS, 
    CAST(ST.UDATE AS DATE) AS DATA_MODIFICACAO_STATUS, 
    CAST(ST.UTIME AS TIME) AS HORA_MODIFICACAO_STATUS, 
    ST.INACT AS INDICADOR_STATUS_INATIVO, 
    ST.CHIND AS INDICADOR_TIPO_MODIFICACAO, 
    ST._CHANGE_TIME AS DATA_HORA_ATUALIZACAO_ODS
FROM DADOS_BRUTOS.K_PM_MANUTENCAO.JSTO AS ESQUEMA
INNER JOIN DADOS_BRUTOS.K_PM_MANUTENCAO.JCDS AS ST
    ON ESQUEMA.OBJNR = ST.OBJNR
INNER JOIN DADOS_BRUTOS.K_PM_MANUTENCAO.TJ30T AS STD
    ON ST.STAT = STD.ESTAT
    AND ESQUEMA.STSMA = STD.STSMA
WHERE ESQUEMA.OBJNR IN (
    SELECT O2.OBJNR
    FROM DADOS_BRUTOS.K_PM_MANUTENCAO.AUFK AS O2
    INNER JOIN DADOS_BRUTOS.K_PM_MANUTENCAO.JCDS AS ST2
     ON O2.OBJNR = ST2.OBJNR
    WHERE O2.WERKS = '1400'
      AND O2.AEDAT > '{dataFormatada}'
      AND ST2.UDATE > '{dataFormatada}'
)
AND STD.TXT04 IN ('FELE', 'FINT', 'FENG', 'FCAL', 'FMEC', 'DLEX', 'FAPL', 'CPXC', 'PTTC')
AND ST.UDATE > '{dataFormatada}'
"""
df = pd.read_sql(strSQL, connection)

df

C:\Users\D25E\AppData\Local\Temp\ipykernel_38196\2522780230.py:59: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strSQL, connection)


,OBJETO_ORDEM,STATUS,TEXTO_STATUS,USUARIO_MODIFICACAO_STATUS,DATA_MODIFICACAO_STATUS,HORA_MODIFICACAO_STATUS,INDICADOR_STATUS_INATIVO,INDICADOR_TIPO_MODIFICACAO,DATA_HORA_ATUALIZACAO_ODS
0,OR002028117629,CPXC,Complexidade de planej Baixa,A1VQ,2025-02-04,17:04:30,,I,2025-03-17 13:02:24.161
1,OR002029804772,CPXC,Complexidade de planej Baixa,A1VQ,2025-02-14,16:49:13,,U,2025-03-17 13:02:24.158
2,OR002029807148,CPXC,Complexidade de planej Baixa,RXTS,2025-01-03,14:30:11,X,U,2025-03-17 13:02:24.138
3,OR002029804488,CPXC,Complexidade de planej Baixa,A1VQ,2025-02-14,16:55:53,,U,2025-03-17 13:02:24.171
4,OR002029834745,CPXC,Complexidade de planej Baixa,A1VQ,2025-02-14,16:54:28,,U,2025-03-17 13:02:24.179
...,...,...,...,...,...,...,...,...,...
12146,OR002029512064,FELE,Falta Serviço Elétrica,F5QC,2025-04-14,08:09:42,,U,2025-04-14 08:09:48.933
12147,OR002029512064,FELE,Falta Serviço Elétrica,F5QC,2025-04-14,08:09:21,X,U,2025-04-14 08:09:35.660
12148,OR002029512064,FELE,Falta Serviço Elétrica,C03P,2025-03-25,13:01:21,,U,2025-03-25 13:01:24.556
12149,OR002029512064,FELE,Falta Serviço Elétrica,ASB5,2025-04-15,07:49:02,X,U,2025-04-15 07:49:05.243


In [17]:
df

,MANDT,AUFPL_VOR,APLZL_VOR,AUFPL_NCH,APLZL_NCH,AOBAR,MIMAX,OBJNR,ZEINH,DAUER,...,SSABD,SSABZ,FEABD,FEABZ,SEABD,SEABZ,DAUTM,AUCRE,DAUERMAX,AENNR
0,400,0000000073,00000001,0000000073,00000002,NF,,NR00000000000000026485,,0.0,...,20020620,240000,20020620,240000,20020620,240000,0,,0.0,
1,400,0000000074,00000001,0000000074,00000002,NF,,NR00000000000000026487,,0.0,...,20021029,240000,20021029,240000,20021029,240000,0,,0.0,
2,400,0000000075,00000001,0000000075,00000002,NF,,NR00000000000000026770,,0.0,...,20030914,240000,20020823,240000,20030914,240000,0,,0.0,
3,400,0000000076,00000001,0000000076,00000002,NF,,NR00000000000000026501,,0.0,...,20020828,240000,20020829,240000,20020828,240000,0,,0.0,
4,400,0000000078,00000001,0000000078,00000002,NF,,NR00000000000000026507,,0.0,...,20021204,240000,20021205,240000,20021204,240000,0,,0.0,
5,400,0000000079,00000001,0000000079,00000002,NF,,NR00000000000000026512,,0.0,...,20021227,240000,20021227,240000,20021227,240000,0,,0.0,
6,400,0000000222,00000001,0000000222,00000002,NF,,NR00000000000000029214,,0.0,...,20040928,240000,20041011,240000,20040928,240000,0,,0.0,
7,400,0000000223,00000001,0000000223,00000002,NF,,NR00000000000000029220,,0.0,...,20041019,240000,20041019,240000,20041019,240000,0,,0.0,
8,400,0000000624,00000001,0000000624,00000002,NF,,NR00000000000000244355,,0.0,...,20060305,240000,20060305,240000,20060305,240000,0,,0.0,
9,400,0000000884,00000001,0000000884,00000002,NF,,NR00000000000000153856,,0.0,...,20030529,240000,20030529,240000,20030529,240000,0,,0.0,
